#### Imports

In [3]:
import os

### Small Movie Lens Dataset

In [12]:
small_dataset_path = './../../Datasets/movieLenSmall'

#### Ratings

In [13]:
small_ratings_path = os.path.join(small_dataset_path, 'ratings.csv')
small_ratings = sc.textFile(name=small_ratings_path)

Remove the header line from the ratings

In [14]:
small_ratings_header = small_ratings.take(1)[0]

In [15]:
small_ratings_header

'userId,movieId,rating,timestamp'

In [16]:
small_ratings_data = small_ratings.filter(lambda line: line!=small_ratings_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

In [17]:
small_ratings_data.take(3)

[('1', '1', '4.0'), ('1', '3', '4.0'), ('1', '6', '4.0')]

#### Movies

In [18]:
small_movies_path = os.path.join(small_dataset_path, 'movies.csv')
small_movies = sc.textFile(name=small_movies_path)

In [19]:
small_movies_header = small_movies.take(1)[0]

In [20]:
small_movies_header

'movieId,title,genres'

In [21]:
small_movies_data = small_movies.filter(lambda line: line!=small_movies_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1])).cache()

In [22]:
small_movies_data.take(3)

[('1', 'Toy Story (1995)'),
 ('2', 'Jumanji (1995)'),
 ('3', 'Grumpier Old Men (1995)')]

#### Declare Datasets

In [23]:
training_small_ratings, validation_small_ratings_temp, test_small_ratings_temp = small_ratings_data.randomSplit([6,2,2], seed= 400)

In [24]:
validation_small_ratings = validation_small_ratings_temp.map(lambda x: (x[0], x[1]))

In [25]:
test_small_ratings = test_small_ratings_temp.map(lambda x: (x[0], x[1]))

### Training a recommender with Aternarting Least Squares

In [26]:
from pyspark.mllib.recommendation import ALS
import math

#### Setting the training parameters

In [27]:
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

In [28]:
min_error = 1e3
best_rank = -1
best_iteration = -1

In [29]:
for rank in ranks:
    
    model = ALS.train(training_small_ratings, rank, seed=0, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_small_ratings).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_small_ratings_temp.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print (f'For rank {rank} the RMSE is {error}')
    if error < min_error:
        min_error = error
        best_rank = rank

For rank 4 the RMSE is 0.9004611692709928
For rank 8 the RMSE is 0.9035024272363643
For rank 12 the RMSE is 0.9068175717565644


In [30]:
print(f'The best model was derived from rank:{best_rank} with a minimum error of {min_error}')

The best model was derived from rank:4 with a minimum error of 0.9004611692709928


In [31]:
predictions.take(3)

[((232, 45208), 1.4357010972030038),
 ((368, 3272), 3.131348041802032),
 ((610, 52328), 4.125442899653168)]

In [32]:
model = ALS.train(training_small_ratings, rank, seed=5, iterations=iterations,
                  lambda_=regularization_parameter)
predictions = model.predictAll(test_small_ratings).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_small_ratings_temp.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())

In [33]:
print(f'Test set RMSE error: {error}')

Test set RMSE error: 0.918218211778835


#### Making predictions

Loading the movies file:

In [35]:
small_movies_file = os.path.join(small_dataset_path, 'movies.csv')
small_movies = sc.textFile(small_movies_file)
small_movies_header = small_movies.take(1)[0]

In [36]:
small_movies_data = small_movies.filter(lambda line: line!=small_movies_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1],tokens[2])).cache()

In [37]:
small_movies_titles = small_movies_data.map(lambda x: (int(x[0]),x[1]))

In [40]:
small_movies_titles.take(10)

[(1, 'Toy Story (1995)'),
 (2, 'Jumanji (1995)'),
 (3, 'Grumpier Old Men (1995)'),
 (4, 'Waiting to Exhale (1995)'),
 (5, 'Father of the Bride Part II (1995)'),
 (6, 'Heat (1995)'),
 (7, 'Sabrina (1995)'),
 (8, 'Tom and Huck (1995)'),
 (9, 'Sudden Death (1995)'),
 (10, 'GoldenEye (1995)')]

In [146]:
def avg(lst):
    
    return sum(lst)/len(lst)

small_movie_ID_with_ratings_RDD = (small_ratings_data.map(lambda x: (x[1], x[2])).groupByKey()).map(lambda x : (x[0], [float(elem) for elem in list(x[1])]))
small_movie_rating_counts_RDD = small_movie_ID_with_ratings_RDD.map(lambda x: (int(x[0]), (avg(x[1]), len(x[1]))))

In [147]:
small_movie_rating_counts_RDD.take(1)

[(1, (3.9209302325581397, 215))]

Create a fake user with ID 0 and assign some ratings

In [148]:
new_user_ID = 0

# The format of each line is (userID, movieID, rating)
new_user_ratings = [
     (0,1,5), # Toy Story (1995)
     (0,2,5), # Jumanji (1995)
     (0,3,2), # Grumpier Old Men (1995)
     (0,4,1), # Waiting to Exhale (1995)
     (0,5,1), # Father of the Bride Part II (1995)
     (0,6,3), # Heat (1995)
     (0,7,1), # Sabrina (1995)
     (0,8,2), # Tom and Huck (1995)
     (0,9,3) , # Sudden Death (1995)
     (0,10,4) # GoldenEye (1995)
    ]
new_user_ratings_RDD = sc.parallelize(new_user_ratings)

In [149]:
complete_data_with_new_ratings_RDD = small_ratings_data.union(new_user_ratings_RDD)

In [129]:
from time import time

start = time()

new_ratings_model = ALS.train(complete_data_with_new_ratings_RDD, best_rank, seed=0, 
                              iterations=iterations, lambda_=regularization_parameter)

print(f'Model trained in {time() - start} seconds')

Model trained in 1.9609730243682861 seconds


In [150]:
new_user_ratings_ids = map(lambda x: x[1], new_user_ratings) # get just movie IDs
# keep just those not on the ID list (thanks Lei Li for spotting the error!)
new_user_unrated_movies_RDD = (small_movies_data.filter(lambda x: x[0] not in new_user_ratings_ids).map(lambda x: (new_user_ID, x[0])))

# Use the input RDD, new_user_unrated_movies_RDD, with new_ratings_model.predictAll() to predict new ratings for the movies
new_user_recommendations_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)

Show the extracted recommendations

In [153]:
new_user_recommendations_rating_title_and_count_RDD.take(2)

[(69720, ((3.845444347349426, 'Hood of Horror (2006)'), (4.5, 1))),
 (3240, ((3.0143997470382344, '"Big Tease'), (3.25, 2)))]

In [154]:
new_user_recommendations_rating_RDD = new_user_recommendations_RDD.map(lambda x: (x.product, x.rating))
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_RDD.join(small_movies_titles).join(small_movie_rating_counts_RDD)
new_user_recommendations_rating_title_and_count_RDD.take(5) 

[(69720, ((3.845444347349426, 'Hood of Horror (2006)'), (4.5, 1))),
 (3240, ((3.0143997470382344, '"Big Tease'), (3.25, 2))),
 (98160, ((1.2273797070898143, 'Nature Calls (2012)'), (1.5, 1))),
 (1320,
  ((2.8746905379925787, 'Alien³ (a.k.a. Alien 3) (1992)'),
   (3.148936170212766, 47))),
 (2040,
  ((3.0644923806019424, '"Computer Wore Tennis Shoes'),
   (2.857142857142857, 7)))]

In [155]:
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_title_and_count_RDD.map(lambda r: (r[1][0][1], r[1][0][0], r[1][1]))

Take top recommendations with more than 10 reviews:

In [161]:
top_movies = new_user_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2][1]>=10).takeOrdered(10, key=lambda x: -x[1])

In [168]:
for elem in top_movies:
    
    print('-----------')
    print(f'{elem[0]}, Rate {elem[2][0]} from {elem[2][1]} users, with predicted review: {elem[1]}')

-----------
Spotlight (2015), Rate 4.157894736842105 from 19 users, with predicted review: 4.660183307844523
-----------
"Hours, Rate 3.7 from 15 users, with predicted review: 4.627460194139214
-----------
His Girl Friday (1940), Rate 4.392857142857143 from 14 users, with predicted review: 4.586486231333916
-----------
Louis C.K.: Live at the Beacon Theater (2011), Rate 4.3 from 10 users, with predicted review: 4.569074157860395
-----------
"Hunt, Rate 4.3 from 10 users, with predicted review: 4.555907820631949
-----------
Three Colors: Red (Trois couleurs: Rouge) (1994), Rate 4.21875 from 16 users, with predicted review: 4.553413213374373
-----------
The Artist (2011), Rate 4.045454545454546 from 11 users, with predicted review: 4.518346156097071
-----------
Grave of the Fireflies (Hotaru no haka) (1988), Rate 4.15625 from 16 users, with predicted review: 4.515764321252634
-----------
Three Colors: Blue (Trois couleurs: Bleu) (1993), Rate 3.8958333333333335 from 24 users, with predict

### MovieLens Big dataset 

In [24]:
full_dataset_path = '/Users/alexandros.ferles/Desktop/Personal_Github/Movie_Recommendation_System/Datasets/MovieLensFull'
ratings_path = os.path.join(full_dataset_path, 'ratings.csv')
ratings = sc.textFile(name=ratings_path)

In [51]:
ratings_header = ratings.take(1)[0]
ratings_data = ratings.filter(lambda line: line!=ratings_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2]))).cache()
movies_titles = ratings_data.map(lambda x: (int(x[0]),x[1]))

In [26]:
training_RDD, test_RDD = ratings_data.randomSplit([7, 3])

In [27]:
complete_model = ALS.train(training_RDD, best_rank, seed=5, 
                           iterations=iterations, lambda_=regularization_parameter)

#### Create a user with reviews

In [28]:
new_user_ID = 0

# The format of each line is (userID, movieID, rating)
new_user_ratings = [
     (0,260,4), # Star Wars (1977)
     (0,1,3), # Toy Story (1995)
     (0,16,3), # Casino (1995)
     (0,25,4), # Leaving Las Vegas (1995)
     (0,32,4), # Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
     (0,335,1), # Flintstones, The (1994)
     (0,379,1), # Timecop (1994)
     (0,296,3), # Pulp Fiction (1994)
     (0,858,5) , # Godfather, The (1972)
     (0,50,4) # Usual Suspects, The (1995)
    ]
new_user_ratings_RDD = sc.parallelize(new_user_ratings)

In [42]:
new_user_ratings_RDD.collect()

[(0, 260, 4),
 (0, 1, 3),
 (0, 16, 3),
 (0, 25, 4),
 (0, 32, 4),
 (0, 335, 1),
 (0, 379, 1),
 (0, 296, 3),
 (0, 858, 5),
 (0, 50, 4)]

Merge with the old ratings:

In [46]:
complete_ratings = ratings_data.union(new_user_ratings_RDD)

In [44]:
full_model = ALS.train(complete_ratings, best_rank, seed=5, 
                              iterations=iterations, lambda_=regularization_parameter)

In [49]:
new_user_ratings_ids = map(lambda x: x[1], new_user_ratings) # get just movie IDs
new_user_unrated_movies_RDD = (complete_ratings.filter(lambda x: x[0] not in new_user_ratings_ids).map(lambda x: (new_user_ID, x[0])))
new_user_recommendations_RDD = full_model.predictAll(new_user_unrated_movies_RDD)

In [ ]:
new_user_recommendations_rating_RDD = new_user_recommendations_RDD.map(lambda x: (x.product, x.rating))
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_RDD.join(movies_titles).join(movie_rating_counts_RDD)
new_user_recommendations_rating_title_and_count_RDD.take(3)